In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.load_world("Town04")
spectator = world.get_spectator()

# Functions
def move_spectator_to(transform, distance=1.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None,
                transform=carla.Transform(carla.Location(x=2.0, z=1.5), carla.Rotation(pitch=5)),
                width=800, height=600,
                horizontal_fov=30,
                vertical_fov=20,
                range=100,
                sensor_tick=0,
                points_per_second=1500):
    
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('range', str(range))
    radar_bp.set_attribute('sensor_tick', str(sensor_tick))
    radar_bp.set_attribute('points_per_second', str(points_per_second))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach_to)
    
    # rad_location = carla.Location(x=2.0, z=1.0)
    # rad_rotation = carla.Rotation(pitch=5)
    # rad_transform = carla.Transform(rad_location,rad_rotation)
    # radar.set_transform(rad_transform)
    return radar

map = world.get_map()
map

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


Constants used in the following scripts.

In [2]:
EGO_VEHICLE_INITIAL_LOCATION = carla.Location(x=280, y=-207.5, z=0.1) # z=0.1 used to make the car not clip with the ground
EGO_VEHICLE_INITIAL_ROTATION = carla.Rotation(yaw=180)

## Spawn ego vehicle

Spawn the ego vehicle in a parking slot, with the spectator facing the back of the vehicle.

In [47]:
pygame.init()
screen = pygame.display.set_mode((200, 100))

# print(world.get_blueprint_library().filter("vehicle.mitsubishi.fusorosa"))

# obstacle_vehicle = spawn_vehicle(pattern="vehicle.mitsubishi.fusorosa")
# obstacle_transform = obstacle_vehicle.get_transform()
# obstacle_transform.location = carla.Location(x=286, y=-208.5, z=0.1)
# obstacle_transform.rotation = carla.Rotation(yaw=90)
# obstacle_vehicle.set_transform(obstacle_transform)

# obstacle_vehicle2 = spawn_vehicle(pattern="vehicle.carlamotors.carlacola")
# obstacle2_transform = obstacle_vehicle2.get_transform()
# obstacle2_transform.location = carla.Location(x=282, y=-210.5, z=0.1)
# obstacle2_transform.rotation = carla.Rotation(yaw=180)
# obstacle_vehicle2.set_transform(obstacle2_transform)

reverse = False

h_fov = 130
range = 40
yaw = 90 + h_fov // 2
vehicle = spawn_vehicle()
right_radar = spawn_radar(attach_to=vehicle,
                          transform=carla.Transform(carla.Location(x=-1.8, y=0.75, z=0.5), carla.Rotation(pitch=3, yaw=yaw)),
                          horizontal_fov=h_fov,
                          range=range)
left_radar = spawn_radar(attach_to=vehicle,
                          transform=carla.Transform(carla.Location(x=-1.8, y=-0.75, z=0.5), carla.Rotation(pitch=3, yaw=-yaw)),
                          horizontal_fov=h_fov,
                          range=range)

points = np.zeros((600, 800, 4), dtype=np.uint8)



def radar_callback(radar_data):
    if not reverse:
        return
    # velocity_range = 7.5 # m/s
    velocity_range = 0.8
    current_rot = radar_data.transform.rotation
    for detect in radar_data:
        azi = math.degrees(detect.azimuth)
        alt = math.degrees(detect.altitude)
        # The 0.25 adjusts a bit the distance so the dots can
        # be properly seen
        fw_vec = carla.Vector3D(x=detect.depth - 0.25)
        carla.Transform(
            carla.Location(),
            carla.Rotation(
                pitch=current_rot.pitch + alt,
                yaw=current_rot.yaw + azi,
                roll=current_rot.roll)).transform(fw_vec)

        def clamp(min_v, max_v, value):
            return max(min_v, min(value, max_v))

        # detection
        detection_threshold = 3 # meter
        if detect.depth < detection_threshold:

            # draw detected points
            norm_velocity = detect.velocity / velocity_range # range [-1, 1]
            r = int(clamp(0.0, 1.0, 1.0 - norm_velocity) * 255.0)
            g = int(clamp(0.0, 1.0, 1.0 - abs(norm_velocity)) * 255.0)
            b = int(abs(clamp(- 1.0, 0.0, - 1.0 - norm_velocity)) * 255.0)

            if norm_velocity < 0:#r > 0 and b == 0 and g == 0:

                # print(norm_velocity)
                # if norm_velocity < -0.002:
                #     r = 255
                # elif norm_velocity < -0.001:
                #     r = 170
                # else:
                #     r = 80
                print("Fermati brigante maometto!!")
                

                world.debug.draw_point(
                    radar_data.transform.location + fw_vec,
                    size=0.075,
                    life_time=0.06,
                    persistent_lines=False,
                    color=carla.Color(r, g, b))

        # draw radars
        world.debug.draw_point(radar_data.transform.location,
                               size=0.075,
                                life_time=0.06,
                                persistent_lines=False,
                                color=carla.Color(2, 0, 255))
        
        

right_radar.listen(lambda image: radar_callback(image))
left_radar.listen(lambda image: radar_callback(image))

# rear_camera = spawn_camera(attach_to=vehicle, transform=carla.Transform(carla.Location(x=3, z=4), carla.Rotation(pitch=-20, yaw=180)), width=window_size[0], height=window_size[1])

# Set the vehicle position
obstacle_transform = vehicle.get_transform()
obstacle_transform.location = EGO_VEHICLE_INITIAL_LOCATION
obstacle_transform.rotation = EGO_VEHICLE_INITIAL_ROTATION
vehicle.set_transform(obstacle_transform)

rear_camera_transform = vehicle.get_transform()
rear_camera_transform.location = obstacle_transform.location + carla.Location(x=1, y=0, z=5)
# rear_camera_transform.rotation = carla.Rotation(yaw=180)
move_spectator_to(rear_camera_transform)

running = True

try:
    while running:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                key = event.key
                print(f"Key pressed: {pygame.key.name(key)}")
                if key == pygame.QUIT or key == pygame.K_ESCAPE or key == pygame.K_q:
                    running = False
                elif key == pygame.K_r:
                    reverse = not reverse
                    control = carla.VehicleControl(reverse=reverse)
                    vehicle.apply_control(control)
                elif key == pygame.K_w:
                    print("Going " + ("backward" if reverse else "foreward"))
                    control = carla.VehicleControl(throttle=0.4, reverse=reverse)
                    vehicle.apply_control(control)
                elif key == pygame.K_s:
                    control = carla.VehicleControl(throttle=0, brake=1)
                    vehicle.apply_control(control)
        world.tick()
        pygame.display.flip()
finally:
    pygame.quit()
    right_radar.destroy()
    left_radar.destroy()
    vehicle.destroy()
    obstacle_vehicle.destroy()
    # obstacle_vehicle2.destroy()


Key pressed: r
Key pressed: q


RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [ ]:
for actor in world.get_actors().filter("vehicle.*"):
    actor.destroy()

ego_vehicle = spawn_vehicle()
time.sleep(1)
print(ego_vehicle.get_transform().location)
print(world.get_map().get_spawn_points()[0])

# Set the vehicle position
obstacle_transform = ego_vehicle.get_transform()
obstacle_transform.location = EGO_VEHICLE_INITIAL_LOCATION
obstacle_transform.rotation = EGO_VEHICLE_INITIAL_ROTATION
ego_vehicle.set_transform(obstacle_transform)

# Set the spectator camera
rear_camera_transform = ego_vehicle.get_transform()
rear_camera_transform.location = obstacle_transform.location + carla.Location(x=2)
rear_camera_transform.rotation = carla.Rotation(yaw=180)
move_spectator_to(rear_camera_transform)


Spawn some traffic vehicles

In [ ]:
def spawn_vehicle_right_lane(
        x=285, y=-227.5,
        time_to_live=4,
        spawn_delay=0,
        start_delay=0):
    time.sleep(spawn_delay)
    traffic_vehicle = spawn_vehicle()

    traffic_vehicle_transform = traffic_vehicle.get_transform()
    traffic_vehicle_transform.location = carla.Location(x=x, y=y)
    traffic_vehicle_transform.rotation = carla.Rotation(yaw=92) # with yaw=90 it's not aligned with the parking slots
    traffic_vehicle.set_transform(traffic_vehicle_transform)

    time.sleep(start_delay)
    traffic_vehicle.apply_control(carla.VehicleControl(throttle=1))

    time.sleep(time_to_live)
    traffic_vehicle.destroy()

spawn_vehicle_right_lane()

Spawn a pedestrian

In [ ]:
for actor in world.get_actors().filter("walker.pedestrian.*"):
    actor.destroy()
def spawn_pedestrians_right_lane(
        x=283.5,
        y=-213,
        time_to_live=10,
        spawn_delay=0,
        start_delay=0):
    time.sleep(spawn_delay)

    blueprint_library = world.get_blueprint_library()
    walker_bp = random.choice(blueprint_library.filter('walker.pedestrian.*'))
    spawn_points = world.get_map().get_spawn_points()
    spawn_point = random.choice(spawn_points)
    pedestrian = world.spawn_actor(walker_bp, spawn_point)

    pedestrian_transform = pedestrian.get_transform()
    pedestrian_transform.location = carla.Location(x=x, y=y)
    pedestrian_transform.rotation = carla.Rotation(yaw=-92)
    pedestrian.set_transform(pedestrian_transform)

    time.sleep(start_delay)
    pedestrian.apply_control(carla.WalkerControl(direction=carla.Vector3D(0, 1, 0), speed=1))

    time.sleep(time_to_live)
    pedestrian.destroy()

spawn_pedestrians_right_lane()

# while True:
#     move_spectator_to(pedestrian.get_transform())
#     world.tick()
#     time.sleep(0.01)


In [ ]:
ego_vehicle = spawn_vehicle()

time.sleep(1)

#target_transform = target_vehicle.get_transform()
#target_transform.location += carla.Location(x=100)
#target_vehicle.set_transform(target_transform)

ego_vehicle = spawn_vehicle()
obstacle_transform = ego_vehicle.get_transform()
obstacle_transform.location = carla.Location(x=283, y=-206)
ego_vehicle.set_transform(obstacle_transform)

camera = spawn_camera(attach_to=ego_vehicle)

video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

camera.listen(lambda image: camera_callback(image))

# target_vehicle.set_autopilot(True)

ego_vehicle.apply_control(carla.VehicleControl(throttle = 1))

# target_velocity = carla.Vector3D(x=20) # Set the target velocity to 20 m/s
# ego_vehicle.set_target_velocity(target_velocity)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('RGB Camera', video_output)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    ego_vehicle.destroy()
    ego_vehicle.destroy()